In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
total = pd.read_csv('./data/2013-2018년_가스공급량과_기온2.csv')

In [3]:
sub = pd.read_csv('./data/sample_submission.csv')

In [4]:
total = total.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', '연월일'])

In [5]:
total

,시간,구분,공급량,year,month,day,weekday,기온
0,1,0,2497.129,2013,1,1,1,-6.4
1,1,1,2169.093,2013,1,1,1,-6.4
2,1,2,226.178,2013,1,1,1,-6.4
3,1,3,1434.516,2013,1,1,1,-6.4
4,1,4,3272.837,2013,1,1,1,-6.4
...,...,...,...,...,...,...,...,...
368083,24,2,237.911,2018,12,31,0,-2.9
368084,24,3,1422.478,2018,12,31,0,-2.9
368085,24,4,3534.260,2018,12,31,0,-2.9
368086,24,5,3982.757,2018,12,31,0,-2.9


In [6]:
sub['일자'] = sub['일자|시간|구분'].str.split().str[0]
sub['시간'] = sub['일자|시간|구분'].str.split().str[1]
sub['구분'] = sub['일자|시간|구분'].str.split().str[2]
d_map = {}
for i, d in enumerate(sub['구분'].unique()):
    d_map[d] = i
sub['구분'] = sub['구분'].map(d_map)
sub['일자'] = pd.to_datetime(sub['일자'])
sub['year'] = sub['일자'].dt.year
sub['month'] = sub['일자'].dt.month
sub['day'] = sub['일자'].dt.day
sub['weekday'] = sub['일자'].dt.weekday

In [7]:
sub

,일자|시간|구분,공급량,일자,시간,구분,year,month,day,weekday
0,2019-01-01 01 A,0,2019-01-01,01,0,2019,1,1,1
1,2019-01-01 02 A,0,2019-01-01,02,0,2019,1,1,1
2,2019-01-01 03 A,0,2019-01-01,03,0,2019,1,1,1
3,2019-01-01 04 A,0,2019-01-01,04,0,2019,1,1,1
4,2019-01-01 05 A,0,2019-01-01,05,0,2019,1,1,1
...,...,...,...,...,...,...,...,...,...
15115,2019-03-31 20 H,0,2019-03-31,20,6,2019,3,31,6
15116,2019-03-31 21 H,0,2019-03-31,21,6,2019,3,31,6
15117,2019-03-31 22 H,0,2019-03-31,22,6,2019,3,31,6
15118,2019-03-31 23 H,0,2019-03-31,23,6,2019,3,31,6


In [8]:
X = total.loc[:, ['시간', 'day', 'month']]
y = total['기온']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [11]:
col = ['month', 'day', '시간']
temp_X = sub[col]

In [12]:
temp_pred = xgb.predict(temp_X)
temp_pred

array([-1.0970039, -0.9044036,  3.787046 , ..., -1.6482983, -1.6482983,
       -1.6482983], dtype=float32)

In [13]:
sub['기온'] = temp_pred
sub

,일자|시간|구분,공급량,일자,시간,구분,year,month,day,weekday,기온
0,2019-01-01 01 A,0,2019-01-01,01,0,2019,1,1,1,-1.097004
1,2019-01-01 02 A,0,2019-01-01,02,0,2019,1,1,1,-0.904404
2,2019-01-01 03 A,0,2019-01-01,03,0,2019,1,1,1,3.787046
3,2019-01-01 04 A,0,2019-01-01,04,0,2019,1,1,1,7.227159
4,2019-01-01 05 A,0,2019-01-01,05,0,2019,1,1,1,10.627350
...,...,...,...,...,...,...,...,...,...,...
15115,2019-03-31 20 H,0,2019-03-31,20,6,2019,3,31,6,-1.648298
15116,2019-03-31 21 H,0,2019-03-31,21,6,2019,3,31,6,-1.648298
15117,2019-03-31 22 H,0,2019-03-31,22,6,2019,3,31,6,-1.648298
15118,2019-03-31 23 H,0,2019-03-31,23,6,2019,3,31,6,-1.648298


In [14]:
X_sub = sub[["시간", 'month','구분', '기온']]

In [15]:
X_sub

,시간,month,구분,기온
0,01,1,0,-1.097004
1,02,1,0,-0.904404
2,03,1,0,3.787046
3,04,1,0,7.227159
4,05,1,0,10.627350
...,...,...,...,...
15115,20,3,6,-1.648298
15116,21,3,6,-1.648298
15117,22,3,6,-1.648298
15118,23,3,6,-1.648298


In [16]:
X = total[["시간", 'month', '구분','기온']]
y = total['공급량']
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
import pycaret
from pycaret.regression import *

In [26]:
# create a conda environment
conda create --name yourenvname python=3.8

# activate conda environment
conda activate yourenvname

# install pycaret
pip install pycaret

# create notebook kernel connected with the conda environment
python -m ipykernel install --user --name yourenvname --display-name "display-name"

SyntaxError: invalid syntax (Temp/ipykernel_19560/1593698269.py, line 2)

In [25]:
exp_clf = setup(X_train, y_train = 'Class variable')

TypeError: setup() got an unexpected keyword argument 'y_train'

In [23]:
top5 = compare_models(sort='Accuracy', n_select=5)

NameError: name '_all_models' is not defined